In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow_hub as hub
import time
from tqdm import tqdm
import math
import seaborn as sns
import json

In [2]:
path = os.getcwd()
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed = hub.load(path+'\\model')

Vectorisation

In [3]:
df = pickle.load(open(path+'\\cleaned_data_no_null.sav', 'rb'))

In [5]:
df.head()

,sha,source_x,title,doi,pmcid,abstract,publish_time,authors,journal,institution,country,body_text
0,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,clinical features of culture-proven mycoplasma...,10.1186/1471-2334-1-6,PMC35282,objective: this retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,{},{},mycoplasma pneumoniae is a common cause of upp...
1,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,{University of Alabama at Birmingham},{USA},since its discovery as a biological messenger ...
2,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,surfactant protein-d and pulmonary host defense,10.1186/rr19,PMC59549,surfactant protein-d (sp-d) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,{Washington University School of Medicine},{USA},surfactant protein-d (sp-d) is a member of the...
3,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,endothelin-1 (et-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,{},{},from xenopus laevis. eta receptors in normal l...
4,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,respiratory syncytial virus (rsv) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,{National Institutes of Health},{USA},rsv and pvm are viruses of the family paramyxo...


In [8]:
sentences = np.array(df['abstract'])

In [12]:
abstract_vec = np.zeros(shape=(1, 512))
groups_num = 20 # depends on GPU memery (8G set to 20 works fine for me)
for i in tqdm(range(math.ceil(len(sentences)/groups_num))):
    if (i < math.ceil(len(sentences)/groups_num)-1):
        vec = sentences[(i*groups_num):((i+1)*groups_num)]
    else:
        vec = sentences[(i*groups_num):len(sentences)]
    embed_vectors = embed(vec).numpy()
    abstract_vec = np.append(abstract_vec, embed_vectors, axis=0)
    
abstract_vec = np.delete(abstract_vec, 0, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████| 8021/8021 [20:29<00:00,  6.52it/s]


In [13]:
pickle.dump(sentences, open('./abstract_data.pkl', 'wb'))
pickle.dump(abstract_vec, open('./abstract_embed.pkl', 'wb'))

Model

In [15]:
abstract_data = pickle.load(open('./abstract_data.pkl', 'rb'))
abstract_embed = pickle.load(open('./abstract_embed.pkl', 'rb'))

In [16]:
def query(text):
    corr = np.inner(embed([text]).numpy(), abstract_embed)
    return abstract_data[np.argmax(corr)]

In [17]:
query('What is the best covid treatment?')

'apart from the treatment options, protocols and preventive measures against covid, an essential step prior to the treatment is its quick and accurate diagnosis. there are several diagnostic protocols prevailing currently for the identification of covid infection. the gold standard available diagnosis is based on the identification of pathogen by rtpcr analysis . this involves extraction of the genetic material of the virus followed by its amplification using specific primers and identification.'